<p style="font-family: Arial; font-size:3em;color:black;"> Session 5 - Lab Exercise 9</p>

Students in this Group:
- Albornoz Cambiaso, Francisco
- Anicio Pereira Junior, Jose
- Cespedes Acosta, Braulio Jose
- Fernandes Lemos, Debora
- Jami, Alimul Hasan
- Kakkar, Himanshu

In [109]:
# For this example, we will use K-Means Clustering Project database from Kaggle (https://www.kaggle.com/faressayah/k-means-clustering-private-vs-public-universities)
# We actually have the labels for this data set, but we will NOT use them for the KMeans clustering algorithm, since that is an unsupervised learning algorithm.
# As we will shortly see, we have a data frame with 777 observations on 18 variables.

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


%matplotlib inline

In [110]:
df = pd.read_csv('College_Data')
df.columns

Index(['College', 'Private', 'Apps', 'Accept', 'Enroll', 'Top10perc',
       'Top25perc', 'F.Undergrad', 'P.Undergrad', 'Outstate', 'Room.Board',
       'Books', 'Personal', 'PhD', 'Terminal', 'S.F.Ratio', 'perc.alumni',
       'Expend', 'Grad.Rate'],
      dtype='object')

In [111]:
df['Grad.Rate']['Cazenovia College'] = 100
#Removing college as a index column
df = df.drop('College', axis=1)
#Removing feature to confirm the impact in the model performance
#df = df.drop('Top10perc', axis=1)
#df = df.drop('Top25perc', axis=1)
#df = df.drop('S.F.Ratio', axis=1)
#df = df.drop('perc.alumni', axis=1)
#df = df.drop('Grad.Rate', axis=1)
# Try removing various columns (features) from the dataset and examine if it improves/degrades your K-Means model performance, or it may have little impact.
# Report 10 cases where you removed one or more features and indicate how it impacted the model performance.
# Case 0: No feature removed. Result: Without removing any feature we get an accuracy of 52%(0.52) 
# Case 1: Removed feature 'Accept'. Result: Removing Accept, decrease the accuracy to a 44%(0.44)
# Case 2: Removed feature 'Book'. Result: Removing Book, increase  the accuracy to a 53%(0.53)
# Case 3: Removed feature 'Book' and 'Outstate'. Result: This combination has a significant impact, reducing the accuracy to a 21%
# Case 4: Removed feature 'Room.Board', 'Terminal', 'Personal', 'Accept' and 'PhD' . Result: Execution will result in an accuracy of 38% 
# Case 5: Removed feature 'Book', 'Terminal', 'PhD', 'Accept', 'Top10perc'. Result: Accuracy was reduced to a 19%, meaning that one or more of those feature has a significant impact in clusterization process. 
# Case 6: Removed feature 'Book', 'Terminal', restore features 'PhD', 'Accept'. Result: Removing those features has a significant negative impact in the accuracy, with a 19%
# Case 7: Removed feature with low numeric values: [perc.alumni, S.F.Ratio, Top25perc, Top10perc]. Result: By reducing features with small weight from the dataframe, We have noticed a significant improvement in the accuracy: 80%, Although, we have observed cases where the accuracy drop to a 20%
# Case 8: Removed features [perc.alumni, S.F.Ratio, Top25perc, Top10perc, S.F.Ratio, Grad.Rate] . Result: By removing these features It made the model unreliable, in the first run one could get 20% and in other an 80%.
# Case 9: Removed feature 'Top10perc'. Result: Removing this feature made the model unstable, the accuracy values jump from 20% to 80% in some instances.
# Case 10: Leaving all the features. Results: Removing feature to this model made the results unstable and unpredicatable, by leaving the features we can get a consistent 50% accuracy. 


C:\Users\TUF\AppData\Local\Temp\ipykernel_2092\2945803996.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Grad.Rate']['Cazenovia College'] = 100


In [112]:
df.head()

,Private,Apps,Accept,Enroll,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Yes,1660,1232,721,52,2885,537,7440,3300,450,2200,70,78,18.10,12,7041,60
1,Yes,2186,1924,512,29,2683,1227,12280,6450,750,1500,29,30,12.20,16,10527,56
2,Yes,1428,1097,336,50,1036,99,11250,3750,400,1165,53,66,12.90,30,8735,54
3,Yes,417,349,137,89,510,63,12960,5450,450,875,92,97,7.70,37,19016,59
4,Yes,193,146,55,44,249,869,7560,4120,800,1500,76,72,11.90,2,10922,15


In [113]:
pd.set_option('display.float', '{:.2f}'.format)
df.describe()

,Apps,Accept,Enroll,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
count,777.00,777.00,777.00,777.00,777.00,777.00,777.00,777.00,777.00,777.00,777.00,777.00,777.00,777.00,777.00,777.00
mean,3001.64,2018.80,779.97,55.80,3699.91,855.30,10440.67,4357.53,549.38,1340.64,72.66,79.70,14.09,22.74,9660.17,65.46
std,3870.20,2451.11,929.18,19.80,4850.42,1522.43,4023.02,1096.70,165.11,677.07,16.33,14.72,3.96,12.39,5221.77,17.18
min,81.00,72.00,35.00,9.00,139.00,1.00,2340.00,1780.00,96.00,250.00,8.00,24.00,2.50,0.00,3186.00,10.00
25%,776.00,604.00,242.00,41.00,992.00,95.00,7320.00,3597.00,470.00,850.00,62.00,71.00,11.50,13.00,6751.00,53.00
50%,1558.00,1110.00,434.00,54.00,1707.00,353.00,9990.00,4200.00,500.00,1200.00,75.00,82.00,13.60,21.00,8377.00,65.00
75%,3624.00,2424.00,902.00,69.00,4005.00,967.00,12925.00,5050.00,600.00,1700.00,85.00,92.00,16.50,31.00,10830.00,78.00
max,48094.00,26330.00,6392.00,100.00,31643.00,21836.00,21700.00,8124.00,2340.00,6800.00,103.00,100.00,39.80,64.00,56233.00,118.00


In [114]:
from sklearn.cluster import KMeans
kmeans = KMeans(2)

In [115]:
kmeans.fit(df.drop('Private', axis=1))

D:\Developments\Python\Math\pythonProject\.venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=2)

In [116]:
df['Private'] = df.Private.astype("category").cat.codes

In [117]:
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [128]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

scalar = StandardScaler()

X = df.drop('Private', axis=1)
y = df.Private

X = scalar.fit_transform(X)

kmeans = KMeans(2)
kmeans.fit(X)

print(kmeans.cluster_centers_)

print(accuracy_score(y, kmeans.labels_))
print(confusion_matrix(y, kmeans.labels_))
print(classification_report(y, kmeans.labels_))

[[-0.28071654 -0.30541441 -0.33544463 -0.054861   -0.34312881 -0.23178088
   0.08598096  0.03022996 -0.04208633 -0.14459703 -0.10783599 -0.10723275
  -0.09619221  0.10228305  0.00904708  0.05837137]
 [ 1.64952019  1.79464753  1.97110826  0.32236904  2.01626134  1.36196909
  -0.50523325 -0.17763446  0.24730373  0.84966752  0.63365571  0.630111
   0.56523563 -0.60102609 -0.05316162 -0.34299635]]
0.16602316602316602
[[114  98]
 [550  15]]
              precision    recall  f1-score   support

           0       0.17      0.54      0.26       212
           1       0.13      0.03      0.04       565

    accuracy                           0.17       777
   macro avg       0.15      0.28      0.15       777
weighted avg       0.14      0.17      0.10       777


D:\Developments\Python\Math\pythonProject\.venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
